In [31]:
import mlflow
from mlflow.models import infer_signature
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# mlflow server --host 127.0.0.1 --port 8080

In [32]:
# Load the Iris dataset
iris = load_iris()

X = pd.DataFrame(iris.data, columns=iris.feature_names)  # Convert to DataFrame
y = iris.target

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average= 'weighted')
f1 = f1_score(y_test, y_pred, average= 'weighted')

metrics = {'accuracy': accuracy,
           'precision': precision,
           'recall' : recall,
           'f1' : f1
}

## MLFLOW TRACKING

In [33]:

# set tracking server

import mlflow.models

mlflow.set_tracking_uri("http://127.0.0.1:8080")

# create a new experiment

mlflow.set_experiment("MLflow_demo")

# start mlflow run

with mlflow.start_run(nested=True):
    
    mlflow.log_params(params)
    
    mlflow.log_metrics(metrics)
    
    mlflow.set_tag("training info", "basic LR model")
                                    
# infer the model signature

signature = mlflow.models.infer_signature(X_train, lr.predict(X_train))

🏃 View run resilient-loon-860 at: http://127.0.0.1:8080/#/experiments/304988124568526473/runs/ee4fe78e67f04687ba92dd9481ad2e44
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/304988124568526473


In [34]:
# log the model

# Log the model
model_info = mlflow.sklearn.log_model(
    sk_model=lr,
    signature=signature,
    artifact_path='iris_model',
    input_example=X_train,
    registered_model_name='tracking_demo_model'
) 

Registered model 'tracking_demo_model' already exists. Creating a new version of this model...
2025/03/27 20:18:10 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking_demo_model, version 5
Created version '5' of model 'tracking_demo_model'.


In [35]:
#  Load the model as a Python Function (pyfunc) and use it for inference

loaded_model= mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)


iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
73,6.1,2.8,4.7,1.2,1,1
18,5.7,3.8,1.7,0.3,0,0
118,7.7,2.6,6.9,2.3,2,2
78,6.0,2.9,4.5,1.5,1,1


## Explicitly Defining a Signature
If infer_signature() doesn’t capture the exact schema you need, you can manually define the signature using mlflow.types.Schema.



In [36]:
from mlflow.models.signature import ModelSignature
from mlflow.types import Schema, ColSpec

# Define input schema
input_schema = Schema([
   ColSpec('float', col) for col in X.columns
])

# Define output schema
output_schema = Schema([ColSpec("integer", "predicted_class")])

# Create the signature
custom_signature = ModelSignature(inputs=input_schema, outputs=output_schema)

print(custom_signature)

mlflow.sklearn.log_model(
    sk_model=lr,
    artifact_path="my_model",
    signature=custom_signature,  # Custom schema
    input_example=X_train[:5]    # Example input
)



inputs: 
  ['sepal length (cm)': float (required), 'sepal width (cm)': float (required), 'petal length (cm)': float (required), 'petal width (cm)': float (required)]
outputs: 
  ['predicted_class': integer (required)]
params: 
  None

